In [531]:
import pandas as pd
pd.set_option('max_column',100)
pd.set_option('max_row',100)

In [532]:

def date_columns_from_statement(columns):
    return [date_col for date_col in columns if re.match('[0-9]{4}-[0-9]{2}-[0-9]{2}',date_col)]


In [806]:

import pickle
from collections import OrderedDict 
  

with open('../data/debug/statement_dict.pickle', 'rb') as f:
    statement_dict = pickle.load(f)

with open('../data/debug/needs_adjustment.pickle', 'rb') as f:
    needs_adjustment = pickle.load(f)
            
with open('../data/debug/tag_map.pickle', 'rb') as f:
    tag_map = pickle.load(f)

with open('../data/debug/timeseries_df.pickle', 'rb') as f:
    timeseries_df = pickle.load(f)
        
freq_list = ['instant','qtd','6mtd','9mtd','ytd']
    
def apply_tag_map_to_needs_adjustment_dict(needs_adjustment, tag_map):
    tag_map_needs_adjustment_intersection = set(tag_map.keys()).intersection(needs_adjustment.keys())

    for matched_xbrl_tag in tag_map_needs_adjustment_intersection:
        for xbrl_tag in tag_map[matched_xbrl_tag]:
            needs_adjustment[xbrl_tag].update(needs_adjustment[matched_xbrl_tag])
            del(needs_adjustment[matched_xbrl_tag])
            needs_adjustment[xbrl_tag] = OrderedDict(sorted(needs_adjustment[xbrl_tag].items(),reverse=True))
    return needs_adjustment

def period_type_to_months(period_type):
    if period_type == 'ytd':
        return 12
    elif period_type == '9mtd':
        return 9
    elif period_type == '6mtd':
        return 6
    elif period_type == 'qtd':
        return 3
    else:
        return None
    

In [807]:
tag_map_needs_adjustment_intersection = set(tag_map.keys()).intersection(needs_adjustment.keys())



In [808]:
tag_map_needs_adjustment_intersection

{'us-gaap_incomelossfromcontinuingoperationsbeforeincometaxesminorityinterestandincomelossfromequitymethodinvestments',
 'us-gaap_salesrevenuenet'}

In [809]:
for matched_xbrl_tag in tag_map_needs_adjustment_intersection:
    for xbrl_tag in tag_map[matched_xbrl_tag]:
        needs_adjustment[xbrl_tag].update(needs_adjustment[matched_xbrl_tag])
        del(needs_adjustment[matched_xbrl_tag])
        needs_adjustment[xbrl_tag] = OrderedDict(sorted(needs_adjustment[xbrl_tag].items(),reverse=True))
return needs_adjustment


KeyError: 'us-gaap_revenuefromcontractwithcustomerexcludingassessedtax'

In [ ]:
needs_adjustment = apply_tag_map_to_needs_adjustment_dict(needs_adjustment, tag_map)

    

for xbrl_tag in needs_adjustment:
    for date_col in needs_adjustment[xbrl_tag]:


        months_in_metric = period_type_to_months(needs_adjustment[xbrl_tag][date_col])
        month_index = timeseries_df.columns.get_loc(date_col)
        tag_index = timeseries_df.index.get_loc(xbrl_tag)

        i = 1
        while (months_in_metric > 3):


            if timeseries_df.columns[month_index+i] in needs_adjustment[xbrl_tag]:
                months_in_metric = months_in_metric - period_type_to_months(needs_adjustment[xbrl_tag][timeseries_df.columns[month_index+i]])
            else:
                months_in_metric = months_in_metric - 3

            if months_in_metric >= 3:
                timeseries_df.iloc[tag_index,month_index] = timeseries_df.iloc[tag_index,month_index] - timeseries_df.iloc[tag_index,month_index+i]


            else:
                print("ERROR: ")


            i += 1
            #break
        

#timeseries_df.loc[xbrl_tag,:]

In [796]:
xbrl_tag = 'us-gaap_depreciationdepletionandamortization'
timeseries_df.loc[xbrl_tag,:]

2020-06-27    2.752000e+09
2020-03-28    2.786000e+09
2019-12-28    2.816000e+09
2019-09-28    3.179000e+09
2019-06-29    2.933000e+09
2019-03-30    3.040000e+09
2018-12-29    3.395000e+09
2018-09-29    2.754000e+09
2018-06-30    2.665000e+09
2018-03-31    2.739000e+09
2017-12-30    2.745000e+09
2017-09-30    2.484000e+09
2017-07-01    2.354000e+09
2017-04-01    2.332000e+09
2016-12-31    2.987000e+09
2016-09-24    2.548000e+09
2016-06-25    2.526000e+09
2016-03-26    2.477000e+09
2015-12-26    2.954000e+09
2015-09-26    3.119000e+09
2015-06-27    3.084000e+09
2015-03-28    2.479000e+09
2014-12-27    2.575000e+09
2014-09-27    1.969000e+09
2014-06-28    1.946000e+09
2014-03-29    1.887000e+09
2013-12-28    2.144000e+09
2013-09-28    1.783000e+09
2013-06-29    1.694000e+09
2013-03-30    1.692000e+09
2012-12-29    1.588000e+09
2012-09-29    9.810000e+08
2012-06-30    8.350000e+08
2012-03-31    7.400000e+08
2011-12-31    7.210000e+08
2011-09-24    5.430000e+08
2011-06-25    4.810000e+08
2